In [7]:
import numpy as np
import pandas as pd

In [12]:
def create_design_mat(df, tot_width=False):
    """
    Completes the preprocessing of DataFrame to create the design matrix for telescope position problem 
    
            output:
            design_mat
            pinv
            y
            beta 
    
    """

    # ea. row must contain (x, y, z) for ea. telescope (except E1, the zero point)
    # and all 30 POP settings for ea. telescope
    import pdb; pdb.set_trace()
    telescopes = ["E1", "W2", "W1", "S2", "S1", "E2"]
    theta = df.elevation
    phi = df.azimuth
    S = np.array([np.cos(theta)*np.sin(phi), np.cos(theta)*np.cos(phi), np.sin(theta)]).T
    k = 0
    
    tot_unique_pops = []
    
    width = 15

    for telescope in telescopes: 

        pop_tel_1 = df[df.tel_1 == telescope].pop_1
        pop_tel_2 = df[df.tel_2 == telescope].pop_2
        tel_1_unique_pops = np.unique(pop_tel_2)
        tel_2_unique_pops = np.unique(pop_tel_1)
        tel_unique_pops = np.union1d(tel_1_unique_pops, tel_2_unique_pops)
        tot_unique_pops.append(tel_unique_pops)
        width += len(tel_unique_pops)
    
    if tot_width: width = 195
        
    design_mat = np.zeros((len(df), width))

    for i, telescope in enumerate(telescopes):

        # for ea. new telescope must jump 3 places for (x, y, z)

        if telescope != "E1":  # keep 'E1' as the zero point

            design_mat[:, 3*  i - 3: 3 + 3 * i - 3] += S * np.where(
                df["tel_1"] == telescope, 1, 0
            ).reshape(-1, 1)
            design_mat[:, 3 * i - 3: 3 + 3 * i - 3] -= S * np.where(
                df["tel_2"] == telescope, 1, 0
            ).reshape(-1, 1)

        for pop in tot_unique_pops[i]:
            
            # not working because E1 isn't in the list of telescopes
            design_mat[:, 15 + k] += np.where(
                (df["pop_1"] == pop) & (df["tel_1"] == telescope), 1, 0
            )  # add when it is tel_1
            design_mat[:, 15 + k] -= np.where(
                (df["pop_2"] == pop) & (df["tel_2"] == telescope), 1, 0
            )  # subtract when it is the tel_2
            
            k += 1 

    y = df["cart_2"].values - df["cart_1"].values

    pinv = np.linalg.pinv(design_mat)
    
    beta = pinv @ y

    return design_mat, pinv, y, beta 

In [11]:
def remove_outlier():
    pass

In [6]:
def parameter_uncertainty():
    pass 